In [5]:
import csv
import random
import networkx as nx

# Configuration Data
TOT_HOSTS = 9
HOSTS = ['client', 'stamper', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7']

# Features to be generated
FEATURES = [
    "CPU Availability",
    "Storage Availability",
    "Network Bandwidth",
    "Overall Health",
    "Capacity"
]

# Simulate a network topology using networkx
def build_topology():
    graph = nx.Graph()

    # Topology connections based on config
    connections = [
        ('client', 'switch-1'), ('stamper', 'switch-1'),
        ('h1', 'switch-1'), ('h2', 'switch-1'), ('h3', 'switch-1'), ('h4', 'switch-1'),
        ('h5', 'switch-1'), ('h6', 'switch-1'), ('h7', 'switch-1'),
        ('switch-1', 'switch-2'),
        ('switch-2', 'h1'), ('switch-2', 'h2'), ('switch-2', 'h3'), ('switch-2', 'h4'),
        ('switch-2', 'h5'), ('switch-2', 'h6'), ('switch-2', 'h7'), ('switch-2', 'stamper'), ('switch-2', 'client')
    ]

    for node1, node2 in connections:
        graph.add_edge(node1, node2)

    return graph

# Function to calculate pairwise latency and hop count
def calculate_pairwise_metrics(graph):
    pairwise_metrics = {}

    for host in HOSTS:
        shortest_paths = nx.single_source_shortest_path_length(graph, host)
        pairwise_metrics[host] = {
            target: {
                "Latency": round(shortest_paths[target] * random.uniform(1, 3), 2),  # Simulated latency (1-3ms per hop)
                "Hop Count": shortest_paths[target]
            }
            for target in HOSTS if target != host
        }

    return pairwise_metrics

# Function to generate random values for features
def generate_features():
    return {
        "CPU Availability": round(random.uniform(0.4, 1.0), 2),
        "Storage Availability": round(random.uniform(0.4, 1.0), 2),
        "Network Bandwidth": round(random.uniform(10, 100), 2),  # Mbps
        "Overall Health": round(random.uniform(0.5, 1.0), 2),
        "Capacity": random.randint(50, 100),  # Simulated capacity units
    }

# Generate data for all hosts
def generate_node_data(graph, pairwise_metrics):
    node_data = []
    for host in HOSTS:
        features = generate_features()
        node_entry = {"Host": host}
        node_entry.update(features)

        # Add pairwise latency and hop count as columns
        for target in HOSTS:
            if target != host:
                metrics = pairwise_metrics[host][target]
                node_entry[f"Latency_to_{target}"] = metrics["Latency"]
                node_entry[f"HopCount_to_{target}"] = metrics["Hop Count"]

        node_data.append(node_entry)
    return node_data

# Write data to CSV file
def write_to_csv(filename, data):
    fieldnames = ["Host"] + FEATURES

    # Dynamically add pairwise metric columns
    for host in HOSTS:
        for target in HOSTS:
            if target != host:
                fieldnames.append(f"Latency_to_{target}")
                fieldnames.append(f"HopCount_to_{target}")

    with open(filename, "w", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerows(data)

# Main
if __name__ == "__main__":
    filename = "node_features_with_pairwise_metrics.csv"
    topology = build_topology()
    pairwise_metrics = calculate_pairwise_metrics(topology)
    node_data = generate_node_data(topology, pairwise_metrics)
    write_to_csv(filename, node_data)
    print(f"CSV file '{filename}' has been generated with node features and pairwise metrics.")


CSV file 'node_features_with_pairwise_metrics.csv' has been generated with node features and pairwise metrics.
